<h1>Word2vec</h1>

Preference

[1] word2vec Parameter Learning Explained.


# 0. CBOW

## 0.0 One-word Context CBOW

### Input

One-hot variable $\boldsymbol{x}(n\_V, 1)$, assuming $x_k=1; x_{k'}=0, k' \ne k$, represents the $k$-th word of the vocabulary of size $n\_V$.

### Hidden 

One hidden layer $h$. $W(n\_V, n\_h)$ is the weight matrix connecting the input and hidden layer.

Given an one word context $w_I$, the 

> $h = W^T \boldsymbol{x}$

> $= W^T_{(k, \centerdot)}$

> $:= v^T_{w_I}$

The value of hidden layer $h(n\_h, 1)$ can be defined as the represention vector of the input context word $w_I$.

### Output

The output layer is a softmax value list of each word in the vocabulary.

$W'(n\_h, n\_V)$ is the weight matrix connecting the hidden and output layer. 

The unnomalized $j$-the value of output $u_j$ is, 

> $u_j = v'^T_{w_j} h$

$v'_{w_j}(n\_h, 1)$ is the $j$-th column of $W'$.

+ Softmax

For input context word $w_I$ and output word $w_j$, the conditional posterior distribution is, 

> $p(w_j|w_I) = \frac{\exp(u_j)}{\sum_{j'=1}^V \exp(u_j')}$

> $= \frac{\exp(v'^T_{w_j} v_{w_I})}
{\sum_{j'}^{V} \exp(v'^T_{w_{j'}} v_{w_I})} = y_j$

$y_j$ is the $j$-th value of the output layer.

### Objective function

For one training sample, for input context word $w_I$ and the actual output word $w_O$, the training objective is to maximize the conditional posterior distribution.

> $\max p(w_O|w_I) = \max y_{j^*}$

> $=\max \log y_{j^*}$

> $= \max u_{j^*} - \log \sum_{j'=1}^V \exp (u_{j'})$

> $:= -J$

Thus, $J = - \log p(w_O|w_I)$ is our loss function.

### SGD

We need to learn to adjust $W$ and $W'$ which are actually the input and output vectors of words separately.

+ $W'$

> $\frac{\partial{J}}{\partial{u_j}} = y_j - \mathbb{1}(j=j^*)
:= e_j$

> $\frac{\partial{J}}{\partial{w^{'}_{ij}}} = 
\frac{\partial{J}}{\partial{u_j}} \centerdot 
\frac{\partial{u_j}}{\partial{w^{'}_{ij}}}
= e_j \centerdot h_i$

Thus, 

> ${w^{'}_{ij}}^{(new)} = {w^{'}_{ij}}^{(old)} - \eta \centerdot e_j \centerdot h_i$

Or, 

> ${v^{'}_{w_j}}^{(new)} = {v^{'}_{w_j}}^{(old)} - \eta \centerdot e_j \centerdot \boldsymbol{h}$

+ $W$

> $\frac{\partial{J}}{\partial{w_{ki}}} = 
\frac{\partial{J}}{\partial{h_i}} \centerdot
\frac{\partial{h_i}}{\partial{w_{ki}}}$

> $\frac{\partial{J}}{\partial{h_i}} = 
\sum_{j=1}^{n\_V}
\frac{\partial{J}}{\partial{u_j}} \centerdot
\frac{\partial{u_j}}{\partial{h_i}}$

> $\frac{\partial{u_j}}{\partial{h_i}} = w^{'}_{ij}$

> $\frac{\partial{h_i}}{\partial{w_{ki}}} = x_k$

> $h_i = \sum_{k=1}^{n\_V} x_k \centerdot w_{ki}$

$\Rightarrow$

> $\frac{\partial{J}}{\partial{w_{ki}}} = 
(\sum_{j=1}^{n\_V} e_j \centerdot w^{'}_{ij}) \centerdot x_k  $

Thus, 

> ${w_{ki}}^{(new)} = {w_{ki}}^{(old)} - \eta (\sum_{j=1}^{n\_V} e_j \centerdot w^{'}_{ij}) x_k $

Or, 

> ${v_{w_I}}^{(new)} = {v_{w_I}}^{(old)} - \eta EH^T$

## 0.1 Multi-word Context CBOW

+ Input layer, multi-word
Compared to one-word context CBOW, the context here are $C$ words $\{\boldsymbol{x_1}, \boldsymbol{x_2}, ..., \boldsymbol{x_C} \}$.

+ Hidden layer, averaged
The average value of the projections of $C$ input context words is defined as the value hidden layer.

> $h = \frac{1}{C} \sum_{i=1}^C W^T \boldsymbol{x_i}$

+ Update weights, $W_1$

The update rule for $W_1$ keeps the same, except that the value of $h$ is changed.

> ${v^{'}_{w_j}}^{(new)} = {v^{'}_{w_j}}^{(old)} - \eta \centerdot e_j \centerdot \boldsymbol{h}$

+ Update weights, $W$

Update weights for $C$ input context words rather than only one word in the One-word Context CBOW.

> ${v_{w_{I, c}}}^{(new)} = {v_{w_{I, c}}}^{(old)} - \frac{1}{C} \centerdot \eta \centerdot EH^T,  c = \{1, 2, ..., C\}$


# 1.0 Skip-Gram

Give only one input word, predict the context output words with one-side size is $C$.

+ Input Layer

Only one input word $\boldsymbol{x}$.

+ Hidden Layer

Like the one word input context in CBOW, the hidden layer $h$ is, 

> $\boldsymbol{h} = W^T \boldsymbol{x}$

For the input word $w_I$, the value of hidden layer is, 

> $\boldsymbol{h} = W^T_{(w_I, )} := v^T_{w_I}$

+ Output Layer

For the $C$ output context words, 

> $p(w_{c, j} = w_{O, c}|w_I) = y_{c, j}$
> $= \frac{\exp(u_{c, j})}{\sum_{j=1}^{n\_V} \exp(u_{j'})}$

> $u_{c, j} = u_j = {v'}^T_{w_j} \centerdot \boldsymbol{h}, c = \{1, 2, ..., C\}$

+ Loss function

> $J = - \log p(w_{O, 1}, w_{O, 2}, ..., w_{O, C}| w_I) $

> $= - \log \prod_{c=1}^C p(w_{O, c}|w_I)$

> $= - \log \prod_{c=1}^C \frac{\exp(u_{c, j^*_c})}{\sum_{j'=1}^{n\_V}\exp(u_{j'})}$

> $= - \sum_{c=1}^C u_{j^*_c} + C \centerdot \log 
\sum_{j'=1}^{n\_V} \exp(u_{j'})$

+ Update Weights, $W_1$

> $\frac{\partial{J}}{\partial{u_{c, j}}} = 
C \centerdot y_j - 1 := EI_j$

> $\frac{\partial{J}}{\partial{w^{'}_{ij}}} = 
\frac{\partial{J}}{\partial{u_j}} \centerdot
\frac{\partial{u_j}}{\partial{w^{'}_{ij}}} = 
EI_j \centerdot h_i$

> ${w^{'}_{ij}}^{(new)} = {w^{'}_{ij}}^{(old)} - 
\eta \centerdot EI_j \centerdot h_i$

Or, 

> ${v^{'}_{w_j}}^{(new)} = {v^{'}_{w_j}}^{(old)} - \eta \centerdot EI_j \centerdot \boldsymbol{h}$

+ Update Weights, W

> $\frac{\partial{J}}{\partial{h_i}} = 
\sum_{j=1}^{n\_V} \frac{\partial{J}}{\partial{u_j}} \centerdot 
\frac{\partial{u_j}}{\partial{h_i}} = 
\sum_{j=1}^{n\_V} EI_j \centerdot w^{'}_{ij} := EH_i$

> $\frac{\partial{J}}{\partial{w_{ki}}} = 
\frac{\partial{J}}{\partial{h_i}} \centerdot
\frac{\partial{h_i}}{\partial{w_{ki}}} = EH_i \centerdot x_k$

> ${w_{ki}}^{(new)} = {w_{ki}}^{(old)} - \eta \centerdot EH_i \centerdot x_k$

Or, 

> ${v_{w_I}}^{(new)} = {v_{w_I}}^{(old)} - \eta \centerdot EH^T$